In [1]:
import os

In [2]:
path = "../input/kla1task2/output"

In [3]:
labels =os.listdir(path)

In [4]:
d = {}
for x in labels :
    d[x] = len(os.listdir(path+'/'+ x))

In [5]:
len(d)

In [6]:
d

In [7]:
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import matplotlib.pyplot as plt


In [8]:

filenames = os.listdir("../input/kla1task2/output")


In [9]:
files = []
categories = []
for filename in filenames:
    
    file  = "../input/kla1task2/output/" + filename
    
    for x in os.listdir(file):
        p = file +"//"+ x
        files.append(p)
        categories.append(filename)
#         print(filename,p)
    
df = pd.DataFrame({
    'filename': files,
    'category': categories
})

In [10]:
df.head()

In [11]:
FAST_RUN = False
IMAGE_WIDTH=256
IMAGE_HEIGHT=256

In [12]:
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=1

In [13]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate,MaxPooling2D,Dropout,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, Callback

def cnnblock(layer,filters,filtershape,dr = 0.25,deep = False , pool = True) :
    
    y = Conv2D(filters[0], filtershape, padding='same')(layer)
    y = Activation('relu')(y)
    y = Conv2D(filters[1], filtershape, padding='same')(y)
    y = Activation('relu')(y)
    if pool :
        y = MaxPooling2D(pool_size=(2, 2))(y)
    if deep :
        y = BatchNormalization()(y)
        y = Dropout(dr)(y)
    
    
    return y

In [14]:
inputs = Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))

x = cnnblock(inputs,[32,32],3,pool = True)

x = cnnblock(x,[64,64],3)

x = cnnblock(x,[128,128],3,deep = True)

x = cnnblock(x,[96,96],3,deep = True)

x = cnnblock(x,[64,64],3,deep = True)

x = cnnblock(x,[32,32],3,deep = True)

x = cnnblock(x,[16,16],3,deep = True)

x = cnnblock(x,[16,16],3,deep = True)

x = Flatten()(x)

x = Dense(1000, activation='relu')(x)

x = Dense(300, activation='relu')(x)

x = Dense(28, activation='softmax')(x)

model = Model(inputs, x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.summary()

In [18]:
from sklearn.model_selection import train_test_split
df = df.sample(frac = 1)

plt.figure(figsize = (20,20))
df['category'].value_counts().plot.bar()

In [20]:
total_train = df.shape[0]
total_validate = df.shape[0]
batch_size = 64

In [21]:
t_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

t_generator = t_datagen.flow_from_dataframe(
    df, 
    color_mode="grayscale",
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)


In [23]:
t_generator.class_indices

In [24]:
len(d)

In [25]:
d = t_generator.class_indices


In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
red = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

In [ ]:
epochs=100
modelhistory = model.fit(
    t_generator, 
    epochs=epochs,
    
    steps_per_epoch=len(df)//batch_size,callbacks=[red, checkpoint])
    


In [ ]:
model.save_weights("task2(w).h5")
model.save("task2(m).h5")